In [1]:
import pandas as pd
import numpy as np
import random
import torch
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import Dataset
from rdkit import Chem
import sys
from deepchem.feat.smiles_tokenizer import SmilesTokenizer
from minGPT.metrics import *
from minGPT.dataset import *
from minGPT.pipeline import *

/home/raylei/anaconda3/envs/polygen/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Skipped loading some Tensorflow models, missing a dependency. No module named 'tensorflow'
Skipped loading modules with pytorch-geometric dependency, missing a dependency. No module named 'torch_geometric'
Skipped loading modules with pytorch-geometric dependency, missing a dependency. cannot import name 'DMPNN' from 'deepchem.models.torch_models' (/home/raylei/anaconda3/envs/polygen/lib/python3.8/site-packages/deepchem/models/torch_models/__init__.py)
Skipped loading modules with pytorch-lightning dependency, missing a dependency. No module named 'pytorch_lightning'
Skipped loading some Jax models, missing a dependency. No module named 'jax'


In [2]:
from mingpt.model import GPT

# Data preprocessing
pipeline = minGPT()
data_config = pipeline.get_default_data_config()
data_config.file_path = "minGPT/htp_md.csv"
data_config.block_size = 64

print(data_config)
train_dataset, test_dataset = pipeline.data_preprocessing(data_config)


# Model
model_config = pipeline.get_default_model_config()
model_config.model_type = 'gpt-nano'
model_config.vocab_size = train_dataset.get_vocab_size()
model_config.block_size = train_dataset.get_block_size()
pipeline.load_model(model_config)

# Train
train_config = pipeline.get_default_train_config()

print(train_config.device)
train_config.max_iters = 10000
train_config.ckpt_path = "./minGPT/ckpts/"
# train_config.pretrain = "./ckpts/10000.pt"
## Define call back function
def batch_end_callback(trainer):
    if trainer.iter_num % 100 == 0:
        print(f"iter_dt {trainer.iter_dt * 1000:.2f}ms; iter {trainer.iter_num}: train loss {trainer.loss.item():.5f}, val loss {trainer.loss_val.item():.5f}")

train_config.call_back = batch_end_callback
# loss = pipeline.train(train_config)

# Generate
generate_config = pipeline.get_default_generate_config()
generate_config.ckpt_path = "./minGPT/ckpts/10000.pt"
assert generate_config.task == data_config.task
print(generate_config)

results = pipeline.generate(generate_config)

# Evaluate
print(pipeline.evaluate())

input_col: mol_smiles
length: 5
block_size: 64
train_test_split: (0.8, 0.2)
task: conditional
file_path: minGPT/htp_md.csv

number of parameters: 0.12M
auto
ckpts_path: None
num_samples: 100
temperature: 1.0
task: conditional
ckpt_path: ./minGPT/ckpts/10000.pt

(0.92, 0.64, 0.96, 0.9298245614035088, 0.2607731328038293, 0.7017801632788198)
